In [3]:
### Data Ingestion
import langchain
import langchain_community
import langchain_core
from langchain_core.documents import Document

In [21]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader('../data/text_files/sample.txt',encoding='utf8')
data = loader.load()
print(data)

[Document(metadata={'source': '../data/text_files/sample.txt'}, page_content='Here’s a concise and clear sample introduction about Python you can use:\n\nPython is a high-level, interpreted, and general-purpose programming language known for its simplicity and readability. Created by Guido van Rossum and first released in 1991, Python emphasizes clean and easy-to-understand syntax, making it ideal for beginners while still being powerful for professionals.\n\nPython supports multiple programming paradigms, including procedural, object-oriented, and functional programming. It comes with a vast standard library and a rich ecosystem of third-party packages, enabling developers to work on diverse applications such as:\n\nWeb development (e.g., Django, Flask)\nData science & machine learning (e.g., Pandas, NumPy, TensorFlow)\nAutomation & scripting\nGame development\nDesktop applications\nIts cross-platform nature allows Python programs to run on various operating systems without modificati

In [25]:
from langchain_community.document_loaders import DirectoryLoader
dir_loader = DirectoryLoader(
    '../data/text_files',
    glob='**/*.txt',
    loader_cls=TextLoader,
    loader_kwargs={'encoding':'utf8'},
    show_progress=False
)

data = dir_loader.load()
print(data)

[Document(metadata={'source': '..\\data\\text_files\\sample.txt'}, page_content='Here’s a concise and clear sample introduction about Python you can use:\n\nPython is a high-level, interpreted, and general-purpose programming language known for its simplicity and readability. Created by Guido van Rossum and first released in 1991, Python emphasizes clean and easy-to-understand syntax, making it ideal for beginners while still being powerful for professionals.\n\nPython supports multiple programming paradigms, including procedural, object-oriented, and functional programming. It comes with a vast standard library and a rich ecosystem of third-party packages, enabling developers to work on diverse applications such as:\n\nWeb development (e.g., Django, Flask)\nData science & machine learning (e.g., Pandas, NumPy, TensorFlow)\nAutomation & scripting\nGame development\nDesktop applications\nIts cross-platform nature allows Python programs to run on various operating systems without modific

In [27]:
from langchain_community.document_loaders import PyPDFLoader,PyMuPDFLoader
dir_loader = DirectoryLoader(
    '../data/pdf_files',
    glob='**/*.pdf',
    loader_cls=PyMuPDFLoader,
    show_progress=False
)
data = dir_loader.load() 
print(len(data))


16


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader

loader = TextLoader('../data/text_file.txt', encoding='utf8')
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100
)

chunks = splitter.split_documents(docs)
print(len(chunks))
# print(chunks[0].page_content)
# print(chunks[1].page_content)

2
Here’s a concise and clear sample introduction about Python you can use:

Python is a high-level, interpreted, and general-purpose programming language known for its simplicity and readability. Created by Guido van Rossum and first released in 1991, Python emphasizes clean and easy-to-understand syntax, making it ideal for beginners while still being powerful for professionals.

Python supports multiple programming paradigms, including procedural, object-oriented, and functional programming. It comes with a vast standard library and a rich ecosystem of third-party packages, enabling developers to work on diverse applications such as:
Web development (e.g., Django, Flask)
Data science & machine learning (e.g., Pandas, NumPy, TensorFlow)
Automation & scripting
Game development
Desktop applications
Its cross-platform nature allows Python programs to run on various operating systems without modification. The language’s popularity continues to grow due to its versatility, community suppor

In [17]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyMuPDFLoader('../data/pdf_files/ITR.pdf')
data = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=300
)
chunks = splitter.split_documents(data)

for i,chunk in enumerate(chunks):
    chunk.metadata['chunk_id'] = i
    chunk.metadata['document_name'] = 'ITR.pdf'
    chunk.metadata['doc_type'] = 'tax_document'



In [9]:
import numpy as np 
import chromadb
import uuid
import os
from sentence_transformers import SentenceTransformer
from chromadb.config import Settings
from typing import List,Dict,Any,Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
class EmbeddingManager:
    def __init__(self , model_name:str = 'all-MiniLM-L6-v2'):
        self.model_name = model_name
        self.model = None
        self.load_model()

    def load_model(self):
        try:
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully: {self.model_name}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise e 

    def get_embeddings(self , texts:List[str]) -> np.ndarray:
        try:
            embeddings = self.model.encode(texts, show_progress_bar=True)
            print(f"generated embeddings with shape: {embeddings.shape}")
            return embeddings
        except Exception as e:
            print(f"Error generating embeddings: {e}")
            raise e
        
embedding_manager = EmbeddingManager()
embedding_manager    

Model loaded successfully: all-MiniLM-L6-v2


In [ ]:
class VectorStore:
    def __init__(self ,collection_name: str= "pdf_documents" , persist_directory: str = "../data/vector_store"):
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = chromadb.PersistentClient(path = self.persist_directory)
        self.collection = self.client.get_or_create_collection(name = self.collection_name)
        self.intialize_store()

    def intialize_store(self):
        try:
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path = self.persist_directory)

            self.collection = self.client.get_or_create_collection(
                name = self.collection_name,
                metadata={"description":"Vector store for PDF documents"}
            )    
            print(f"Vector store intialized successfully {self.collection_name}")
            print(f"Existing documents in the store: {self.collection.count()}")
        except Exception as e:
            print(f"Error intializing vector store: {e}")
            raise e    
        
    def add_documents(self , documents: List[Any] , embeddings : np.ndarray):
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents and embeddings must be the same")
        
        print(f"Adding {len(documents)} documents to the vector store")

        ids = []
        metadatas = []
        document_texts =[]
        embedding_list = []

        for i,(doc,embedding) in enumerate(zip(documents , embeddings)):
            doc_id = str(uuid.uuid4())
            ids.append(doc_id)

            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadatas.append(metadata)

            document_texts.append(doc.page_content)

            embedding_list.append(embedding)

            try:
                self.collection.add(
                    ids = ids,
                    metadatas = metadatas,
                    documents = document_texts,
                    embeddings = embedding_list
                )
                print(f"Successfully added {len(documents)} documents to the vector store")
            except Exception as e:
                print(f"Error adding documents to vector store: {e}")
                raise e    
vector_store = VectorStore()
vector_store        

Vector store intialized successfully pdf_documents
Existing documents in the store: 0


In [22]:
text = [doc.page_content for doc in chunks]
embeddings = embedding_manager.get_embeddings(text)
vector_store.add_documents(chunks,embeddings)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches: 100%|██████████| 2/2 [00:02<00:00,  1.07s/it]

generated embeddings with shape: (53, 384)
Adding 53 documents to the vector store
Error adding documents to vector store: Expected document to be a str, got page_content='Instructions for filling ITR-1 SAHAJ 
 
 
 
 
     A.Y. 2019-20 
 
 
General Instructions 
 
These instructions are guidelines to help the taxpayers for filling the 
particulars in Income-tax Return Form-1 for the Assessment Year 2019-20 relating to 
the Financial Year 2018-19. In case of any doubt, please refer to relevant provisions 
of the Income-tax Act, 1961 and the Income-tax Rules, 1962.  
 
1. 
Assessment Year for which this Return Form is applicable 
 
 
This Return Form is applicable for assessment year 2019-20 only, i.e., it 
relates to income earned during the Financial Year 2018-19.  
 
2.   
Who is eligible to use this Return Form 
 
 
This Return Form is to be used by an individual who is a resident other than 
not ordinarily resident,  whose total income for the assessment year  2019-20 does 
not exce

ValueError: Expected document to be a str, got page_content='Instructions for filling ITR-1 SAHAJ 
 
 
 
 
     A.Y. 2019-20 
 
 
General Instructions 
 
These instructions are guidelines to help the taxpayers for filling the 
particulars in Income-tax Return Form-1 for the Assessment Year 2019-20 relating to 
the Financial Year 2018-19. In case of any doubt, please refer to relevant provisions 
of the Income-tax Act, 1961 and the Income-tax Rules, 1962.  
 
1. 
Assessment Year for which this Return Form is applicable 
 
 
This Return Form is applicable for assessment year 2019-20 only, i.e., it 
relates to income earned during the Financial Year 2018-19.  
 
2.   
Who is eligible to use this Return Form 
 
 
This Return Form is to be used by an individual who is a resident other than 
not ordinarily resident,  whose total income for the assessment year  2019-20 does 
not exceed Rs. 50 lakh and who has income under the following heads:-  
 
(a)   
Income from Salary/ Pension; or  
(b) 